In [47]:
import pandas as pd
import numpy as np
from surprise import Reader, Dataset, evaluate, dump
from surprise import SVD, SVDpp, KNNBaseline, KNNWithMeans, CoClustering
import threading
import thread

In [48]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)


In [49]:
def processDataFile(data,counter):
    df_nan = pd.DataFrame(pd.isnull(data.Rating))
    df_nan = df_nan[df_nan['Rating'] == True]
    df_nan = df_nan.reset_index()
    a = zip(df_nan['index'][1:],df_nan['index'][:-1])
    store_df = pd.DataFrame([])
    temp_total_array = np.zeros(((0),1))
    for i,j in a:
        temp_np_array = np.full((i-j,1), counter)
        temp_total_array =  np.concatenate((temp_total_array, temp_np_array))
        counter = counter+1
    remaining =  data.shape[0]- temp_total_array.shape[0]
    temp_np_array = np.full((remaining,1), counter)
    temp_total_array =  np.concatenate((temp_total_array, temp_np_array))
    data['movie_id'] = temp_total_array
    data= data.dropna(thresh =3)
    return (data,counter)

In [50]:
total_data = pd.DataFrame([])

In [51]:
data = pd.read_csv("./input/combined_data_1.txt",  names = ['Cust_Id', 'Rating','date'])
temp_data = processDataFile(data,1)
#print "Counter",1
print "Before",total_data.shape
total_data = pd.concat([total_data,temp_data[0]],ignore_index=True)
print "After",total_data.shape
#print mem_usage(data), mem_usage(total_data), " ", mem_usage(temp_data[0])

Before (0, 0)
After (24053764, 4)


In [18]:
data = pd.read_csv("./input/combined_data_2.txt",  names = ['Cust_Id', 'Rating','date'])
temp_data = processDataFile(data,temp_data[1])
print "Counter",temp_data[1]
print "Before",total_data.shape
total_data = pd.concat([total_data,temp_data[0]],ignore_index=True)
print "After",total_data.shape
#print mem_usage(data), mem_usage(total_data), " ", mem_usage(temp_data[0])

Counter 9209
Before (102062710, 4)
After (129040301, 4)
3154.00 MB
15084.00 MB   3359.00 MB


In [52]:
print "After",total_data.shape


After (24053764, 4)


In [1]:
data = pd.read_csv("./input/combined_data_3.txt",  names = ['Cust_Id', 'Rating','date'])
temp_data = processDataFile(data,temp_data[1])
print "Counter",temp_data[1]
print "Before",total_data.shape
total_data = pd.concat([total_data,temp_data[0]],ignore_index=True)
print "After",total_data.shape


In [6]:
total_data.shape

(24053764, 4)

In [ ]:
data = pd.read_csv("./input/combined_data_4.txt",  names = ['Cust_Id', 'Rating','date'])
temp_data = processDataFile(data,temp_data[1])
print "Counter",temp_data[1]
print "Before",total_data.shape
total_data = pd.concat([total_data,temp_data[0]],ignore_index=True)
print "After",total_data.shape


In [54]:
#del data
sample_df = total_data[:4000]

In [55]:
sample_df.head()

,Cust_Id,Rating,date,movie_id
0,1488844,3.0,2005-09-06,1.0
1,822109,5.0,2005-05-13,1.0
2,885013,4.0,2005-10-19,1.0
3,30878,4.0,2005-12-26,1.0
4,823519,3.0,2004-05-03,1.0


In [56]:
temp = pd.pivot_table(sample_df,index='Cust_Id', columns='movie_id')
# temp = pd.pivot_table(sample_df,'movie_id','Cust_Id'

### Create models as a dictionary below

In [57]:
coclustering = CoClustering()
knnbaseline = KNNBaseline()
knnwithmeans = KNNWithMeans()
svd = SVD()
svdpp = SVDpp()
# modelList = {"coclustering" : coclustering, "knnbaseline" : knnbaseline, "knnwithmeans" : knnwithmeans, "svd" :svd, "svdpp" :svdpp}
modelList = {"coclustering" : coclustering, "svd" :svd, "svdpp" :svdpp}

In [58]:
reader = Reader()
data = Dataset.load_from_df(total_data[['Cust_Id', 'movie_id', 'Rating']].sample(n=500000), reader)
data.split(n_folds=3)
trainset = data.build_full_trainset()

In [59]:
# Start of Probe Data manipulation 
probe = pd.read_csv("./input/probe.txt",  names = ['Cust_Id'])
# temp_data = processDataFile(probe,1)
probe_movie = pd.DataFrame([])
Cust_Id = []
movie_id = []
currentMovie =0
index = 0

for row in probe['Cust_Id']:
    if row[len(row)-1] == ":":
        currentMovie = int(row[:-1])
        #print currentMovie
        continue
    Cust_Id.append(row)
    movie_id.append(currentMovie)
    #print index,row
    index +=1
probe_movie['Cust_Id'] = Cust_Id
probe_movie['movie_id'] = movie_id
#print probe_movie[:30]
# test = total_data.loc[(total_data.Cust_Id.isin(probe_movie.Cust_Id)) & total_data.movie_id.isin(probe_movie.movie_id)]
merged_probe_set = pd.merge(probe_movie,total_data,on=['Cust_Id','movie_id'])
#print merged_probe_set[:30]
# End of Probe Data manipulation 
predictions = pd.DataFrame(0, index=np.arange(len(merged_probe_set)), columns=modelList.keys())

In [60]:
def evaluateTrainAndTestModel(modelList, testData, predictions):
    # Evaluate model
    for model in modelList:
        print "Testing model: ", model
        try:
            evaluate(modelList[model], trainData, measures=['RMSE', 'MAE'])

            # Train model
            modelList[model].train(trainset)

            # Predict Movie ratings and compute error
            cust_id = [];
            pred = [];
            index =1;
            mae_sum = 0;
            mse_sum = 0;
            mse_sum_round = 0;
            rmse_round = 0;
            rmse = 0;
            n = len(merged_probe_set)
            for index,row in testData.iterrows():
                user = row['Cust_Id']
                currentMovie = row['movie_id']
                pred = modelList[model].predict(user, currentMovie, r_ui=row['Rating'], verbose=False)
                predictions[model][index]= pred.est
                diff = row['Rating'] - pred.est
                diffRound = row['Rating'] - round(pred.est)
                mse_sum += diff**2
                mse_sum_round += diffRound**2
                #print(diff)
            mse = mse_sum/n
            rmse = (mse_sum/n)**0.5
            rmse_round = (mse_sum_round/n)**0.5

            # TODO: Try using model.test
            #pred = model.test(testData['Cust_Id'])
            #rmse(pred)
            #dump.dump('./dump_SVD', pred, model)
            #print testData[:20]
            print "MSE: ", mse, " RMSE: ", rmse , " RMSE Roundup:  ", rmse_round
        except MemoryError:
            print "!!! Memory Error occured !!!"
            print "Continuing with next model"
    return predictions

In [ ]:
reader = Reader()

# get just top 100K rows for faster run time
data = Dataset.load_from_df(total_data[['Cust_Id', 'movie_id', 'Rating']][:500000], reader)
data.split(n_folds=3)

In [ ]:
reader = Reader()
data = Dataset.load_from_df(total_data[['Cust_Id', 'movie_id', 'Rating']][:500000], reader)
trainset = data.build_full_trainset()


In [12]:
def runModelsfor():
    for i in range(7,20):
        print "Running for i: ",i
        reader = Reader()
# get just top 100K rows for faster run time
        data = Dataset.load_from_df(total_data[['Cust_Id', 'movie_id', 'Rating']][:i*500000], reader)
        data.split(n_folds=3)
        data = Dataset.load_from_df(total_data[['Cust_Id', 'movie_id', 'Rating']][:i*500000], reader)
        trainset = data.build_full_trainset()
        
        for model in modelList:
            try:
                print "Testing model: ", model
                res = evaluateTrainAndTestModel(modelList, merged_probe_set, predictions)
                print "For i: MSE: ",res[0],"RMSE: ",res[1]
            except MemoryError:
                print "!!! Memory Error occured !!!"
                print "Continuing with next model"
        

In [2]:
runModelsfor()

In [ ]:
evaluateTrainAndTestModel(modelList, merged_probe_set, predictions)


Testing model:  coclustering
Evaluating RMSE, MAE of algorithm CoClustering.

------------
Fold 1
RMSE: 0.9853
MAE:  0.7668
------------
Fold 2
RMSE: 0.9824
MAE:  0.7640
------------
Fold 3
RMSE: 0.9829
MAE:  0.7640
------------
------------
Mean RMSE: 0.9835
Mean MAE : 0.7650
------------
------------
MSE:  1.40667508156  RMSE:  1.18603333914  RMSE Roundup:   1.21828917983
Testing model:  svd
Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.9544
MAE:  0.7478
------------
Fold 2
RMSE: 0.9553
MAE:  0.7486
------------
Fold 3
RMSE: 0.9548
MAE:  0.7480
------------
------------
Mean RMSE: 0.9548
Mean MAE : 0.7481
------------
------------
MSE:  1.1314760371  RMSE:  1.06370862416  RMSE Roundup:   1.10322495319
Testing model:  svdpp
Evaluating RMSE, MAE of algorithm SVDpp.

------------
Fold 1


***Data Munging***
<br> The data is not available as a csv, instead it is a TXT file. There are four files which contain the customer ids and the rating that they gave to the movies. In each of these files there are rows with a number followed by a colon and  the rows that follow this kind of rows have the customer ids with the rating and the date on which the rating was given.
To organize this data we have done the following steps:<br>
1. We find all the indices which have movie ids in them and store them in an array.
2. We zip this array with itself such that the resulting dataframe contains the value of the next index in the second column.
3. We use the resultant dataframe and use the values in the first 2 columns to fill an array with movie ids.
4. We append the new array to dataframe that we obtained from the TXT files.
5. Finally, we drop those rows which have the movie ids with colons in them.

We named this dataframe as total_data.

Similarly for the probe set, we read the probe.txt file and store it in a dataframe. Just like we cleaned the movie data set as mentioned above, we have cleaned the probe data as well. The problem with probe data set is that the rows dont have the actual ratings that the movies got. So, we had to find the actual ratings from total_data dataframe by merging the probe dataframe with the total_data dataframe. We named this dataframe as merged_probe_set.<br>

In this assignment we have used collaborative learning as our modelling method for unsupervised learning.


**What is Collaborative Filtering?**
<br>Collaborative Filtering uses preferences of other users to determine the preference of the user in consideration. For example, if person A likes Movie 1,2 and 3, and person B likes movies 1 and 3, then there is a high probability that Person A and B will have affinity for the same kind of movies. So, we can safely assume that Person A and Person B are liked minded and we can use one person's rating for a particular movie to determine the other person's liking for that movie.
<br>
Formally, Collaborative filtering is the process of filtering for information or patterns using techniques involving collaboration among multiple agents, viewpoints, data sources, etc. <a href="https://en.wikipedia.org/wiki/Collaborative_filtering">Source</a><br>

**We have used the following clustering techniques with their corresponding MSE and RMSE on the probe set: **<br>

1. CoClustering: In Co-Clustering, the users are assigned some clusters and some co-clusters. We calculate the predicion using the sum of the difference of the mean of individual clusters and the difference of the co-clusters.
MSE:  1.28873601364  RMSE:  1.13522509382
2. SVD: the user-rating matrix of movies after an SVD, will decompose into matrices that represents latent user-user features and item-item features, e.g. same gender user, same age-group of users, action movies etc. and many other latent factors involved in the rating behavior that is not apparent from the user-rating matrix.<a href ="https://www.quora.com/Whats-the-difference-between-SVD-and-SVD++">Source</a>
3. SVDpp: This is similar to SVD except, it factors in item-item neighborhood and user-user neighborhood models.


**Prevention of Over-fitting**

Although overfitting is not so common in Unsupervised learning as it is in Supervised learing but there are a certain scenarios where overfitting can occur in Unsupervised learning as well. For example: In the following two scenarios:
1. There are N elements and N cluster, and each element belongs to its own cluster.
2. On the other hand if there is one big cluster which has all the elements. Then also the error rate would be very high.

The module that we used to implement collaborative filtering has a feature in which the training data is divided into folds. The api divides the data into a certain number of folds, lets say 5, and out of these 5 folds it uses 4 folds to train and 1 fold to validate the predictions. This helps us in keeping the overfitting in check.

<br>
***Individual Contributions:***
<br>
Our primary motive from this assignment was learning. So, we did not divide the assignment amongst each other instead all three of us individualy and separately did the assignment and merged the best of all three. <br>

Overall, we believe that the contribution of all members of the team was equal.